### Preamble

In [3]:
# Location of the MSIEVE executable

MSIEVE_LOCATION = "msieve153.exe"

### Generate a message
Message is selected randomly from N

In [6]:
import random

N = 2**128

m_i = random.randint(1, N)

#m_i = 2**128

### Factor using MSIEVE

In [7]:
import subprocess

# MSIEVE_LOCATION refers to the identified location of the GNFS implementation on disk
# -q provides MSIEVE instruction to use a distilled output for the factors identified
# m_i is the previously generated submessage
msieve_factor_output_text = subprocess.run(f"{MSIEVE_LOCATION} -q {m_i}", capture_output=True).stdout

msieve_factor_output_text

b'\r\n277257988463235353807985973445179317120\r\np1: 2\r\np1: 2\r\np1: 2\r\np1: 2\r\np1: 2\r\np1: 2\r\np1: 2\r\np1: 3\r\np1: 3\r\np1: 3\r\np1: 3\r\np1: 3\r\np1: 5\r\np2: 11\r\np5: 30757\r\np28: 5269399946242443683583374303\r\n\r\n'

### Interpret the output of MSIEVE into native python objects

In [8]:
import string
import re

factors = [int(x.split(b":")[1]) for x in filter(lambda x: re.match(b"^p.*",x), re.split(b"\r\n",msieve_factor_output_text))]

factors

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 5,
 11,
 30757,
 5269399946242443683583374303]

### Construct a dictionary of prime factors, indices, count

In [9]:
class DistillationDict(dict):
    def __missing__(self, key):
        return None
    
from collections import Counter
def get_dist_dict(f):
    return DistillationDict(Counter(fac for fac in f))

distillation = get_dist_dict(factors)

In [10]:
distillation

{2: 7, 3: 5, 5: 1, 11: 1, 30757: 1, 5269399946242443683583374303: 1}

In [11]:
# Implicitly assumed we're never trying to find the length of "zero"
def find_int_length(n):
    discovered_length = 0
    while n != 0:
        n = n >> 1
        discovered_length += 1
    return discovered_length

In [12]:
import math

# TODO: Correct implementation, right now find an estimate
def get_prime_index(n):
    return (math.ceil(n/math.log(n)) >> 1)

In [13]:
get_prime_index(2)

1

In [14]:
def seek_missing_sequence(pre_distilled):
    test_kern = 1
    
    while True:
        # Check if the sequence is present in each position
        dist_copy = pre_distilled
        while find_int_length(test_kern) <= find_int_length(dist_copy):
            # Compare the test kernel, and the relevant portion of the distilled_copy if they match, we start over
            print(f"Comparing: {bin(test_kern)} with {find_int_length(test_kern)} and {bin(dist_copy & 2**(find_int_length(test_kern)-1))}")
            if not (test_kern ^ (dist_copy & (2**find_int_length(test_kern)-1))):
                print("found match, next kernel")
                break
            
            if(find_int_length(dist_copy) == find_int_length(test_kern)):
                return test_kern
            else:
                # Check the next position
                dist_copy = dist_copy >> 1
                
        test_kern += 1
                
        

def construct_distilled_message(dist_dict):
        distilled_message = find_int_length(m_i)
        for k in dist_dict.keys():
            estimated_prime_index = get_prime_index(k)
            # Dump prime index
            distilled_message = distilled_message << find_int_length(estimated_prime_index)
            distilled_message = distilled_message | estimated_prime_index
            
            # Dump 
            distilled_message = distilled_message << find_int_length(dist_dict[k])
            distilled_message = distilled_message | dist_dict[k]
            
        return distilled_message

unescaped_message = construct_distilled_message(distillation)

In [15]:
bin(unescaped_message)

'0b1000000011111101101101101110100001100010001001000111100101000111110110010111111101110100000000000000000000000000000000001'

In [71]:
import leb128

def construct_leb128_message(dist_dict):
    leb128_message = bytearray()
    for k in dist_dict.keys():
        leb128_message += leb128.u.encode(k)
        leb128_message += b'\x00'
        leb128_message += leb128.u.encode(dist_dict[k])
        leb128_message += b'\x00'
    return leb128_message[:-1]

In [72]:
z = construct_leb128_message(distillation)
z

bytearray(b'\x02\x00\x01\x00\x05\x00\x01\x00\x89\x01\x00\x01\x00\x85\x1b\x00\x01\x00\xd5\xd2\x08\x00\x01\x00\xc3\x8e\x915\x00\x01\x00\x9b\x85\xb6\xd6E\x00\x01\x00\xd7\x8d\x98\xc6\x9d\xb6\xf7\xf1\xad\xd9\x81\x9d}\x00\x01')

In [17]:
import constriction
import numpy as np

message = np.array(z, dtype=np.int8)

# Define an i.i.d. entropy model (see below for more complex models):
entropy_model = constriction.stream.model.QuantizedGaussian(-50, 50, 3.2, 9.6)

# Let's use an ANS coder in this example. See below for a Range Coder example.
encoder = constriction.stream.stack.AnsCoder()
encoder.encode_reverse(message, entropy_model)

compressed = encoder.get_compressed()
print(f"compressed representation: {compressed}")
print(f"(in binary: {[bin(word) for word in compressed]})")

decoder = constriction.stream.stack.AnsCoder(compressed)
decoded = decoder.decode(entropy_model, 9) # (decodes 9 symbols)
assert np.all(decoded == message)

NameError: name 'z' is not defined

In [18]:
print(f"compression ratio: {find_int_length(unescaped_message)/find_int_length(m_i)}")


compression ratio: 0.9453125


In [20]:
import sys

In [23]:
ratios = []

num_rounds = 0

file_size = 4

while True:
    num_rounds += 1
    N = 2**16
    m_i = random.randint(1, N)
    msieve_factor_output_text = subprocess.run(f"{MSIEVE_LOCATION} -q {m_i}", capture_output=True).stdout
    factors = [int(x.split(b":")[1]) for x in filter(lambda x: re.match(b"^p.*",x), re.split(b"\r\n",msieve_factor_output_text))]
    distillation = get_dist_dict(factors)
    unescaped_message = construct_distilled_message(distillation)
    
    ratios += [find_int_length(unescaped_message)/find_int_length(m_i)]
    sys.stdout.write(f"\rcompression ratio average: {sum(ratios)/len(ratios)} after {num_rounds}, {factors}")
    

compression ratio average: 1.1166232170138464 after 1184, [23, 2543]17, 37]71]1]]09], 3, 7]]

KeyboardInterrupt: 